In [ ]:
Objective: Worldwide numbers, More detail for China and Europe

In [1]:
!pip install plotly

In [63]:
import pandas as pd
import numpy as np

covid_global_df = pd.read_csv('./data/covid_19_data.csv')
#Rename column Country/region to Country
covid_global_df = covid_global_df.rename(columns={'Country/Region':'Country'})

#Replace 'None' as NaN
covid_global_df['Province/State'].replace(to_replace='None', value=np.nan, inplace=True)

covid_global_df.head()

,SNo,ObservationDate,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [125]:
# Sort values in order to have the newest numbers on the top
cases_now_df = covid_global_df.copy(deep=True)

cases_now_df.sort_values('ObservationDate', ascending=False, inplace=True)

# Remove rows with duplicate Country Entries
cases_now_df.drop_duplicates(subset = ['Province/State', 'Country'], keep='first', inplace=True)

#cases_now_df = cases_now_df[cases_now_df['Country'] != 'US']
#cases_now_df = cases_now_df[cases_now_df['Country'] != 'Mainland China']

# Sum of all corona cases within Mainland China and the US. Original data is divided by Province/State
cases_now_df = cases_now_df.append(cases_now_df[(cases_now_df['Country'] == 'Mainland China') |
                                       (cases_now_df['Country'] == 'US')].groupby('Country').sum().reset_index(), sort=False, ignore_index=True)

cases_now_df = cases_now_df[cases_now_df['Confirmed']>0]
cases_now_df = cases_now_df[(cases_now_df['Province/State'].isnull())]

In [126]:
cases_now_df

,SNo,ObservationDate,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
3,3292,03/04/2020,NaN,Belgium,2020-03-04T12:33:03,23.0,0.0,1.0
4,3291,03/04/2020,NaN,Iceland,2020-03-04T19:33:03,26.0,0.0,0.0
5,3290,03/04/2020,NaN,United Arab Emirates,2020-03-03T23:43:02,27.0,0.0,5.0
6,3289,03/04/2020,NaN,India,2020-03-04T12:33:03,28.0,0.0,3.0
7,3288,03/04/2020,NaN,Austria,2020-03-04T19:33:03,29.0,0.0,0.0
...,...,...,...,...,...,...,...,...
165,2664,02/28/2020,NaN,Azerbaijan,2020-02-28T15:03:26,1.0,0.0,0.0
177,265,01/27/2020,NaN,Ivory Coast,1/27/20 23:59,1.0,0.0,0.0
178,213,01/26/2020,NaN,Australia,1/26/20 16:00,4.0,0.0,0.0
181,101085,NaN,NaN,Mainland China,NaN,80271.0,2981.0,49955.0


In [130]:
import plotly as py
import plotly.graph_objects as go
#from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) 

fig = go.Figure(data=go.Choropleth(
    locations = cases_now_df['Country'],
    locationmode = 'country names',
    z = cases_now_df['Confirmed'],
    #text = '%s: %s', cases_now_df['Country'], cases_now_df['Confirmed'],
    colorscale = 'Blues',
    autocolorscale=True,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    #colorbar_tickprefix = '$',
    colorbar_title = 'Confirmed cases',
))

fig.update_layout(
    title_text='Confirmed Corona Cases as of X',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset">\
            Kaggle: Novel Corona Virus 2019 Dataset</a>',
        showarrow = False
    )]
)

fig.show()

In [129]:
cases_now_without_china_df = cases_now_df[cases_now_df['Country']!= 'Mainland China']

fig = go.Figure(data=go.Choropleth(
    locations = cases_now_without_china_df['Country'],
    locationmode = 'country names',
    z = cases_now_without_china_df['Confirmed'],
    #text = '%s: %s', cases_now_df['Country'], cases_now_df['Confirmed'],
    colorscale = 'Blues',
    autocolorscale=True,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Confirmed cases',
))

fig.update_layout(
    title_text='Confirmed Corona Cases as of X (without China)',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset">\
            Kaggle: Novel Corona Virus 2019 Dataset</a>',
        showarrow = False
    )]
)

fig.show()

In [135]:
df_country_date = covid_global_df[covid_global_df['Confirmed']>0].copy(deep=True)
df_country_date = df_country_date.groupby(['ObservationDate', 'Country']).sum().reset_index()
df_country_date

,ObservationDate,Country,SNo,Confirmed,Deaths,Recovered
0,01/22/2020,Japan,36,2.0,0.0,0.0
1,01/22/2020,Macau,21,1.0,0.0,0.0
2,01/22/2020,Mainland China,373,547.0,17.0,28.0
3,01/22/2020,South Korea,38,1.0,0.0,0.0
4,01/22/2020,Taiwan,29,1.0,0.0,0.0
...,...,...,...,...,...,...
1449,03/04/2020,UK,3273,85.0,0.0,8.0
1450,03/04/2020,US,107274,153.0,11.0,8.0
1451,03/04/2020,Ukraine,3391,1.0,0.0,0.0
1452,03/04/2020,United Arab Emirates,3290,27.0,0.0,5.0


In [148]:
import plotly.express as px


fig = px.choropleth(df_country_date,
                    locations='Country',
                    locationmode = 'country names',
                    color='Confirmed',
                    animation_frame='ObservationDate'
                          )
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
Follow instructions to make plotly work here:
    
    https://plot.ly/python/getting-started/#jupyterlab-support-python-35